In [ ]:
from IPython.display import clear_output
!pip install vietocr
clear_output()

In [ ]:
!git clone https://github.com/fcakyon/craft-text-detector.git

Cloning into 'craft-text-detector'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 254 (delta 136), reused 242 (delta 131), pack-reused 0
Receiving objects: 100% (254/254), 1.52 MiB | 26.43 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [ ]:
%cd craft-text-detector

/content/craft-text-detector


In [ ]:
!pwd

/content/craft-text-detector


In [ ]:
!chmod +x vgg16_bn.py

chmod: cannot access 'vgg16_bn.py': No such file or directory


In [ ]:
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from craft_text_detector import Craft

import os
import cv2
import numpy as np
import pandas as pd
import time
import sys
import re
from glob import glob
from collections import Counter
from tqdm import tqdm

import json
import argparse
import requests



In [ ]:
def list_files(in_path):
    img_files = []
    mask_files = []
    gt_files = []
    for (dirpath, dirnames, filenames) in os.walk(in_path):
        for file in filenames:
            filename, ext = os.path.splitext(file)
            ext = str.lower(ext)
            if ext == '.jpg' or ext == '.jpeg' or ext == '.gif' or ext == '.png' or ext == '.pgm':
                img_files.append(os.path.join(dirpath, file))
            elif ext == '.bmp':
                mask_files.append(os.path.join(dirpath, file))
            elif ext == '.xml' or ext == '.gt' or ext == '.txt':
                gt_files.append(os.path.join(dirpath, file))
            elif ext == '.zip':
                continue
    # img_files.sort()
    # mask_files.sort()
    # gt_files.sort()
    return img_files, mask_files, gt_files

In [ ]:
# 1 image
def ocr(img_path, detector, craft):

  img_name = img_path.split('/')[-1].replace('.jpg', '')
  # read image_path
  frame = cv2.imread(img_path)
  frame = frame[0:660, :1280]
  # prediction
  prediction_result = craft.detect_text(frame)
  boxes = prediction_result['boxes']
  # Sorted dt_boxes
  if len(boxes) > 0:
    boxes = sorted(boxes, key = lambda x:x[0][1])
  # each box predict a word, which is at the box
  output = []
  for idx, box in enumerate(boxes):
    try:
       box = np.array(box).astype(np.int32).reshape(-1, 2)
       point1, point2, point3, point4 = box
       x, y, w, h = point1[0],point1[1],point2[0] - point1[0],point4[1]-point1[1]
       crop_img = frame[y:y+h, x:x+w]
       crop_img = Image.fromarray(crop_img)
       s = detector.predict(crop_img)
       output.append(s)
    except:
       output.append('')




  return text, img_name # text of 1 image

In [ ]:
def postprocessing(text, texts, idx):
    # print(text)
    new_string = ' '.join(text)
    # print(new_string)
    texts[idx] = new_string
    # print(text)
    texts[idx] = np.array(texts[idx])
    if text == '':
        num_words = 0
    else:
        num_words = len(text.split(' '))

    return num_words

In [ ]:
def inference(folder_path, detector, craft):

  text_outputs = []
  img_names = []
  number_words = []


  image_paths, _, _ = list_files(folder_path)
  for image_path in image_paths:
       text, img_name = ocr(image_path, detector, craft)
       text_outputs.append(text)
       img_names.append(img_name)


  for i, text_output in enumerate(text_outputs):
        # text_output là output của 1 ảnh
        # text_outputs là output của 1 video
        num_words = postprocessing(text_output, text_outputs, i)
        number_words.append(num_words)

  dataframe = {
        'frame_idx': img_names,
        'text_outputs': text_outputs,
        'num_words': number_words
  }
  df = pd.DataFrame(dataframe)

  # custom path to save result
  base_path = '/content/drive/MyDrive/OCR'
  video_name = folder_path.split('/')[-1]
  path = f'{base_path}/{video_name}.csv'

  df.to_csv(path)

  print(f'Done {video_name}')

In [ ]:
config_vietorc = Cfg.load_config_from_name("vgg_transformer")
config_vietorc['weights'] = '/content/drive/MyDrive/OCR/weight/vgg_transformer.pth'
config_vietorc['cnn']['pretrained']=True
config_vietorc['device'] = 'cuda:0'
config_vietorc['predictor']['beamsearch']=False


detector = Predictor(config_vietorc)
craft = Craft(output_dir=None, crop_type="box", cuda=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:01<00:00, 369MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in 

Craft text detector weight will be downloaded to /root/.craft_text_detector/weights/craft_mlt_25k.pth


Downloading...
From: https://drive.google.com/uc?id=1bupFXqT-VU6Jjeul13XP7yx2Sg5IHr4J
To: /root/.craft_text_detector/weights/craft_mlt_25k.pth
100%|██████████| 83.2M/83.2M [00:00<00:00, 103MB/s]


Craft text refiner weight will be downloaded to /root/.craft_text_detector/weights/craft_refiner_CTW1500.pth


Downloading...
From: https://drive.google.com/uc?id=1xcE9qpJXp4ofINwXWVhhQIh9S8Z7cuGj
To: /root/.craft_text_detector/weights/craft_refiner_CTW1500.pth
100%|██████████| 1.85M/1.85M [00:00<00:00, 167MB/s]


In [ ]:
def join_path(prefix, suffixes):
    # frefix = base_path
    # suffix = 1 list == batches
    for idx, suffix in enumerate(suffixes):
         suffix = os.path.join(prefix, suffix)
         suffixes[idx] = suffix
base_path = '/content/drive/MyDrive/'

#input for inference is folder
folder_path =     #
path = os.path.join(base_path, folder_path)

# inference

inference(path, detector, craft)